In [ ]:
import mlflow

from pyngrok import ngrok
from getpass import getpass

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
import xgboost as xgb
import mlflow
import mlflow.xgboost

# Start mlflow in the background

In [2]:
with mlflow.start_run(run_name="MLflow test"):
    mlflow.log_metric("m1", 2.0)
    mlflow.log_param("p1", "mlflow-test")
# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 8889 &")

# PyNgrok for remote tunnel access

In [3]:
# Terminate open tunnels if any exist
ngrok.kill()

# Login on ngrok.com and get your authtoken from https://dashboard.ngrok.com/auth

In [4]:
# Enter your auth token when the code is running
NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken: ')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok_tunnel = ngrok.connect(addr="8889", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

Enter the ngrok authtoken:  ·················································


MLflow Tracking UI: https://72da26be4cb0.ngrok.io


In [5]:
# Loading iris dataset to prepare test and train
iris = datasets.load_iris()
X = iris.data
Y = iris.target
    
def train():
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=25)
    # enable auto logging in MLflow
    mlflow.xgboost.autolog()
    dtrain = xgb.DMatrix(X_train, label=Y_train)
    dtest = xgb.DMatrix(X_test, label=Y_test)
    with mlflow.start_run():
        # train the XGBoost model
        params = {
            "objective": "multi:softprob",
            "num_class": 3,
            "learning_rate": 1,
            "eval_metric": "mlogloss",
            "seed": 42,
        }
        model = xgb.train(params, dtrain, evals=[(dtrain, "train")])
        # evaluate model
        Y_prob = model.predict(dtest)
        Y_pred = Y_prob.argmax(axis=1)
        loss = log_loss(Y_test, Y_prob)
        acc = accuracy_score(Y_test, Y_pred)
        # log metrics
        mlflow.log_metrics({"log_loss": loss, "accuracy": acc})

In [6]:
train()

[0]	train-mlogloss:0.26308
[1]	train-mlogloss:0.12395
[2]	train-mlogloss:0.05955
[3]	train-mlogloss:0.03595
[4]	train-mlogloss:0.02555
[5]	train-mlogloss:0.02392
[6]	train-mlogloss:0.02289
[7]	train-mlogloss:0.02210
[8]	train-mlogloss:0.02165
[9]	train-mlogloss:0.02124
